# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [2]:
import os
import glob
import psycopg2
import numpy as np
import pandas as pd
import datetime
from sql_queries import *
import json
from typing import Any

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")

In [5]:
filepath = song_files[2]

In [6]:
# df = df.append(pd.DataFrame(data = pd.read_json(filepath, lines=True)), ignore_index=True)
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARVBRGZ1187FB4675A,NaN,,NaN,Gwen Stefani,290.55955,1,SORRZGD12A6310DBC3,Harajuku Girls,2004


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [16]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values.tolist()
song_data

[['SOGDBUF12A8C140FAA', 'Intro', 'AR558FS1187FB45658', 2003, 75.67628],
 ['SORRZGD12A6310DBC3',
  'Harajuku Girls',
  'ARVBRGZ1187FB4675A',
  2004,
  290.55955]]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [23]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [24]:
df

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR8IEZO1187B99055E,NaN,,NaN,Marc Shaiman,149.86404,1,SOINLJW12A8C13314C,City Slickers,2008


In [25]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['AR8IEZO1187B99055E', 'Marc Shaiman', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [26]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [5]:
log_files = get_files("data/log_data")

In [6]:
filepath = log_files[0]

In [7]:
df = pd.read_json(filepath, lines=True) 
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [8]:
df = df[df.page == "NextSong"]
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [10]:
t = pd.to_datetime(df['ts'], unit='ms')
t.head()
len(t)

319

In [11]:
t = t.drop_duplicates(keep="last")
len(t)

319

In [29]:
time_log = t.to_json(orient='values')
# print(json.loads(time_log)[9])
# print(datetime.datetime.fromtimestamp(json.loads(time_log)[5]/1000.0).isocalendar()[1])

'[1543449657796,1543449690796,1543449841796,1543449895796,1543450033796,1543450256796,1543450264796,1543450450796,1543450562796,1543450691796,1543450933796,1543451175796,1543451453796,1543451707796,1543452014796,1543452253796,1543452394796,1543452588796,1543453030796,1543453332796,1543453596796,1543453763796,1543453798796,1543454037796,1543454254796,1543454533796,1543454778796,1543455073796,1543455218796,1543455510796,1543455747796,1543455764796,1543456010796,1543456346796,1543456517796,1543458271796,1543458485796,1543458671796,1543458973796,1543459210796,1543460277796,1543460442796,1543460835796,1543465425796,1543465636796,1543465913796,1543467355796,1543467608796,1543479522796,1543479789796,1543483675796,1543489167796,1543489392796,1543489411796,1543489639796,1543489857796,1543494067796,1543497251796,1543499081796,1543499104796,1543499907796,1543499974796,1543499981796,1543500081796,1543500313796,1543500608796,1543500844796,1543501070796,1543501334796,1543501583796,1543501951796,1543

In [13]:
def clean_csv_value(value: Optional[Any]) -> str:
    if value is None:
        return r'\N'
    return str(value).replace('\n', '\\n')

class StringIteratorIO(io.TextIOBase):
    def __init__(self, iter: Iterator[str]):
        self._iter = iter
        self._buff = ''

    def readable(self) -> bool:
        return True

    def _read1(self, n: Optional[int] = None) -> str:
        while not self._buff:
            try:
                self._buff = next(self._iter)
            except StopIteration:
                break
        ret = self._buff[:n]
        self._buff = self._buff[len(ret):]
        return ret

    def read(self, n: Optional[int] = None) -> str:
        line = []
        if n is None or n < 0:
            while True:
                m = self._read1()
                if not m:
                    break
                line.append(m)
        else:
            while n > 0:
                m = self._read1(n)
                if not m:
                    break
                n -= len(m)
                line.append(m)
        return ''.join(line)
    

In [36]:
time_string_iterator = StringIteratorIO((
    '|'.join(map(clean_csv_value, (
        datetime.datetime.fromtimestamp(log/1000.0),
        datetime.datetime.fromtimestamp(log/1000.0).hour,
        datetime.datetime.fromtimestamp(log/1000.0).day,
        datetime.datetime.fromtimestamp(log/1000.0).isocalendar()[1],
        datetime.datetime.fromtimestamp(log/1000.0).month,
        datetime.datetime.fromtimestamp(log/1000.0).year,
        datetime.datetime.fromtimestamp(log/1000.0).weekday(),
    ))) + '\n'
    for log in json.loads(time_log)
))

'2018-11-29 00:00:57.796000|0|29|48|11|2018|3\n2018-11-29 00:01:30.796000|0|29|48|11|2018|3\n2018-11-29 00:04:01.796000|0|29|48|11|2018|3\n2018-11-29 00:04:55.796000|0|29|48|11|2018|3\n2018-11-29 00:07:13.796000|0|29|48|11|2018|3\n2018-11-29 00:10:56.796000|0|29|48|11|2018|3\n2018-11-29 00:11:04.796000|0|29|48|11|2018|3\n2018-11-29 00:14:10.796000|0|29|48|11|2018|3\n2018-11-29 00:16:02.796000|0|29|48|11|2018|3\n2018-11-29 00:18:11.796000|0|29|48|11|2018|3\n2018-11-29 00:22:13.796000|0|29|48|11|2018|3\n2018-11-29 00:26:15.796000|0|29|48|11|2018|3\n2018-11-29 00:30:53.796000|0|29|48|11|2018|3\n2018-11-29 00:35:07.796000|0|29|48|11|2018|3\n2018-11-29 00:40:14.796000|0|29|48|11|2018|3\n2018-11-29 00:44:13.796000|0|29|48|11|2018|3\n2018-11-29 00:46:34.796000|0|29|48|11|2018|3\n2018-11-29 00:49:48.796000|0|29|48|11|2018|3\n2018-11-29 00:57:10.796000|0|29|48|11|2018|3\n2018-11-29 01:02:12.796000|1|29|48|11|2018|3\n2018-11-29 01:06:36.796000|1|29|48|11|2018|3\n2018-11-29 01:09:23.796000|1|29|4

In [32]:
# 
time_data = (t.tolist(), t.dt.hour.tolist(), t.dt.day.tolist(), t.dt.week.tolist(), t.dt.month.tolist(),t.dt.year.tolist(),t.dt.weekday.tolist())
column_labels = ('timestamp', 'hour', 'day', 'week_of_year', 'month', 'year', 'weekday')

In [33]:
time_dict = {column_labels[i] : time_data[i] for i, _ in enumerate(column_labels)}
time_df = pd.DataFrame.from_dict(time_dict)
# time_df = time_df.astype({'timestamp':'str'})
time_df.head()

,timestamp,hour,day,week_of_year,month,year,weekday
0,2018-11-29 00:00:57.796,0,29,48,11,2018,3
1,2018-11-29 00:01:30.796,0,29,48,11,2018,3
2,2018-11-29 00:04:01.796,0,29,48,11,2018,3
3,2018-11-29 00:04:55.796,0,29,48,11,2018,3
4,2018-11-29 00:07:13.796,0,29,48,11,2018,3


In [34]:
time_df.dtypes

timestamp       datetime64[ns]
hour                     int64
day                      int64
week_of_year             int64
month                    int64
year                     int64
weekday                  int64
dtype: object

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [35]:
for i, row in time_df.head().iterrows():    
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [12]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [20]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

In [21]:
user_df[user_df['userId']==""]
# there is empty userid
# I assummed that this is an error
# So I decided not to put these into the users table

,userId,firstName,lastName,gender,level


In [14]:
# user_df[user_df['userId']==''] = np.nan
user_df.replace('', np.nan, inplace=True)
user_df.dropna(subset=['userId'])
user_df[user_df['userId']==""]

/opt/conda/lib/python3.6/site-packages/pandas/core/frame.py:3798: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


,userId,firstName,lastName,gender,level


In [22]:
user_df.head()

,userId,firstName,lastName,gender,level
0,73,Jacob,Klein,M,paid
1,24,Layla,Griffin,F,paid
2,24,Layla,Griffin,F,paid
3,73,Jacob,Klein,M,paid
4,24,Layla,Griffin,F,paid


In [23]:
user_df = user_df.drop_duplicates(subset = 'userId' , keep="last")
len(user_df)
user_df.head()

,userId,firstName,lastName,gender,level
7,73,Jacob,Klein,M,paid
21,50,Ava,Robinson,F,free
34,24,Layla,Griffin,F,paid
39,54,Kaleb,Cook,M,free
47,14,Theodore,Harris,M,free


In [16]:
from typing import Iterator, Dict, Any

def clean_csv_value(value: Optional[Any]) -> str:
    if value is None:
        return r'\N'
    return str(value).replace('\n', '\\n')

In [37]:
user_log = user_df.to_json(orient='values')
json.loads(user_log)

[['73', 'Jacob', 'Klein', 'M', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['73', 'Jacob', 'Klein', 'M', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['73', 'Jacob', 'Klein', 'M', 'paid'],
 ['73', 'Jacob', 'Klein', 'M', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['50', 'Ava', 'Robinson', 'F', 'free'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 ['24', 'Layla', 'Griffin', 'F', 'paid'],
 

In [38]:
user_string_iterator = StringIteratorIO((
    '|'.join(map(clean_csv_value, (
        log[0],
        log[1],
        log[2],
        log[3],
        log[4],
    ))) + '\n'
    for log in json.loads(user_log)
))

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [18]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [7]:
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Sydney Youngblood,Logged In,Jacob,M,53,Klein,238.07955,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,Ain't No Sunshine,200,1543449657796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
1,Gang Starr,Logged In,Layla,F,88,Griffin,151.92771,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My Advice 2 You (Explicit),200,1543449690796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
2,3OH!3,Logged In,Layla,F,89,Griffin,192.52200,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,My First Kiss (Feat. Ke$ha) [Album Version],200,1543449841796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24
3,RÃÂ¶yksopp,Logged In,Jacob,M,54,Klein,369.81506,paid,"Tampa-St. Petersburg-Clearwater, FL",PUT,NextSong,1.540558e+12,954,The Girl and The Robot,200,1543449895796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",73
4,Kajagoogoo,Logged In,Layla,F,90,Griffin,223.55546,paid,"Lake Havasu City-Kingman, AZ",PUT,NextSong,1.541057e+12,984,Too Shy,200,1543450033796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",24


In [10]:
for index, row in df.head().iterrows():
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        
    print(songid, artistid)

InternalError: current transaction is aborted, commands ignored until end of transaction block


In [14]:
# import numpy as np
# df.replace('', np.nan, inplace=True)
# df.dropna(subset=['userId'], inplace=True)
# df[df['userId']==""]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId


In [19]:
s = ""
for index, row in df.iterrows():
    
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()

    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    if row['userId']=="":
        row['userId'] = None
    
    s += '|'.join(map(clean_csv_value, (
            pd.Timestamp(row[row.index.get_loc("ts")]), 
            row[row.index.get_loc("userId")],
            row[row.index.get_loc("level")],
            songid,  
            artistid,
            row[row.index.get_loc("sessionId")],
            row[row.index.get_loc("location")],  
            row[row.index.get_loc("userAgent")],
            ))) + '\n'
    
    if index == 2:
        break

print(StringIteratorIO(s))
print(s)
#     log = row.to_json(orient='values')
    
#     songplay_string_iterator = StringIteratorIO((
#     '|'.join(map(clean_csv_value, (
#         userdata[0],
#         userdata[1],
#         userdata[2],
#         userdata[3],
#         userdata[4],
#     ))) + '\n'
#     ))

1970-01-01 00:25:43.449657796|73|paid|\N|\N|954|Tampa-St. Petersburg-Clearwater, FL|"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"
1970-01-01 00:25:43.449690796|24|paid|\N|\N|984|Lake Havasu City-Kingman, AZ|"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"
1970-01-01 00:25:43.449841796|24|paid|\N|\N|984|Lake Havasu City-Kingman, AZ|"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"



In [15]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    if row['userId']=="":
        row['userId'] = None

    # insert songplay record
    songplay_data = (pd.Timestamp(row.ts), row.userId, row.level, songid,  artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [16]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.